<a href="https://colab.research.google.com/github/ranpoluv/Recomendaco-de-Pacote-de-Internet/blob/main/projeto_de_extens%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=d0a5cd594bb09ed165b01cb3e50869f8c4aa0f2be8893499961fb1a4a7d1eef0
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt

In [5]:
spark = SparkSession.builder.appName("TelecomRecommender").getOrCreate()

In [6]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt

In [9]:
# Importar a biblioteca Pandas para criar e manipular DataFrames
import pandas as pd

# Dados de clientes
data_customers = {
    'customerId': [1, 2, 3],  # IDs únicos dos clientes
    'name': ['Maria', 'Leia', 'João'],  # Nomes dos clientes
    'age': [25, 30, 35]  # Idades dos clientes
}

# Dados dos pacotes de internet
data_packages = {
    'packageId': [101, 102, 103],  # IDs únicos dos pacotes
    'packageName': ['Basic', 'Standard', 'Premium'],  # Nomes dos pacotes
    'price': [19.99, 29.99, 39.99]  # Preços dos pacotes
}

# Dados de uso (avaliações dos pacotes pelos clientes)
data_usage = {
    'customerId': [1, 1, 2, 3, 3],  # IDs dos clientes que avaliaram os pacotes
    'packageId': [101, 102, 101, 103, 102],  # IDs dos pacotes avaliados
    'rating': [5, 4, 3, 5, 4]  # Avaliações dadas pelos clientes (1 a 5)
}

# Criar DataFrames a partir dos dados
df_customers = pd.DataFrame(data_customers)  # DataFrame para dados dos clientes
df_packages = pd.DataFrame(data_packages)  # DataFrame para dados dos pacotes
df_usage = pd.DataFrame(data_usage)  # DataFrame para dados de uso (avaliações)

# Salvar os DataFrames em arquivos CSV
df_customers.to_csv('/content/customers.csv', index=False)  # Salvar dados dos clientes
df_packages.to_csv('/content/packages.csv', index=False)  # Salvar dados dos pacotes
df_usage.to_csv('/content/usage.csv', index=False)  # Salvar dados de uso (avaliações)

In [10]:
customers = spark.read.csv("/content/customers.csv", header=True, inferSchema=True)
packages = spark.read.csv("/content/packages.csv", header=True, inferSchema=True)
usage = spark.read.csv("/content/usage.csv", header=True, inferSchema=True)

customers.show()
packages.show()
usage.show()

+----------+-----+---+
|customerId| name|age|
+----------+-----+---+
|         1|Maria| 25|
|         2| Leia| 30|
|         3| João| 35|
+----------+-----+---+

+---------+-----------+-----+
|packageId|packageName|price|
+---------+-----------+-----+
|      101|      Basic|19.99|
|      102|   Standard|29.99|
|      103|    Premium|39.99|
+---------+-----------+-----+

+----------+---------+------+
|customerId|packageId|rating|
+----------+---------+------+
|         1|      101|     5|
|         1|      102|     4|
|         2|      101|     3|
|         3|      103|     5|
|         3|      102|     4|
+----------+---------+------+



In [ ]:
# Verificar os dados
usage.show()

# Definir o modelo de recomendação ALS
als = ALS(
    maxIter=10,
    regParam=0.01,
    userCol="customerId",
    itemCol="packageId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

# Ajustar o modelo aos dados
model = als.fit(usage)

# Fazer previsões
predictions = model.transform(usage)

# Avaliar o modelo
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

# Calcular o erro quadrático médio
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

+----------+---------+------+
|customerId|packageId|rating|
+----------+---------+------+
|         1|      101|     5|
|         1|      102|     4|
|         2|      101|     3|
|         3|      103|     5|
|         3|      102|     4|
+----------+---------+------+

Root-mean-square error = 0.003090001346851275


In [ ]:
# Salvar o modelo
model.save("/content/als_model")

# Fazer o download do modelo
from google.colab import files
files.download('/content/als_model')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>